Student: Dorin Doncenco

Use the binarized MNIST database to:

·        Generate images with autoregressive models available in the pytorch library

·        What are the encountered difficulties (initialization…)

·        Comment on the obtained results according to the used settings 


In [1]:
import torch
import torch.nn as nn
import torch.optim
import torch.nn.functional as F
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib
import math


import dataset_loader

%matplotlib inline

mnist_path = "./mnist.pkl.gz"

# the dataset contains 3 splits (train/dev/test),
# each one containing two vectors (pixels and classes)
(train_data_pixels, train_data_classes), \
(dev_data_pixels, dev_data_classes), _ = dataset_loader.load_mnist(mnist_path)

#convert data to pytorch tensors
train_data_pixels = torch.from_numpy(train_data_pixels)
train_data_classes = torch.from_numpy(train_data_classes)
dev_data_pixels = torch.from_numpy(dev_data_pixels)
dev_data_classes = torch.from_numpy(dev_data_classes)

In [2]:
# Masked Autoencoder for Distribution Estimation

class MADE(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_masks=1, natural_ordering=False):
        """
        :param input_size: integer specifying the size of the input
        :param hidden_sizes: a list of integers specifying the size of each hidden layer
        :param num_masks: the number of orderings to use over the inputs. Paper says 1 is enough, more is faster.
        :param natural_ordering: force natural ordering of dimensions, don't use random permutations.
        """
        super().__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.num_masks = num_masks
        self.natural_ordering = natural_ordering

        # define a simple MLP neural net
        self.net = []
        hs = [input_size] + hidden_sizes + [input_size * num_masks]
        for h0, h1 in zip(hs, hs[1:]):
            self.net.extend([
                nn.Linear(h0, h1),
                nn.ReLU(),
            ])
        self.net.pop()  # pop the last ReLU for the output layer
        self.net = nn.Sequential(*self.net)

        # seeds for orders and masks
        self.seed = 0
        self.m = {}
        self.m[-1] = torch.arange(self.input_size)

        # masks is a dictionary of masks
        self.masks = {}
        for i in range(self.num_masks):
            self.masks[i] = {}

    def forward(self, x):
        """
        :param x: Tensor of shape [batch_size, input_size]
        :return: Tensor of shape [batch_size, input_size, num_masks]
        """
        # forward the MADE model
        logits = self.net(x)

        # reshape the output
        logits = logits.view(logits.size(0), self.num_masks, self.input_size)

        # apply softmax to the last dimension
        # and return the probabilities
        return F.softmax(logits, dim=-1)

    def sample(self, x):
        """
        :param x: Tensor of shape [batch_size, input_size]
        :return: Tensor of shape [batch_size, input_size]
        """
        # get the probabilities
        probs = self.forward(x)

        # sample from the distribution
        # and return the sample
        return torch.multinomial(probs[:, -1], 1).squeeze(-1)

In [3]:
#hyperparameters
input_size = 784
hidden_sizes = [512, 512]
num_masks = 1
natural_ordering = False

# create the model
model = MADE(input_size, hidden_sizes, num_masks, natural_ordering)
# define the loss function
loss_function = nn.CrossEntropyLoss()
# define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# number of epochs
num_epochs = 10
# batch size
batch_size = 64

In [5]:
# train the model
for epoch in range(num_epochs):
    # shuffle the data
    permutation = torch.randperm(train_data_pixels.size()[0])
    # get the inputs
    for i in range(0, train_data_pixels.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_data_pixels[indices], train_data_classes[indices]
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(batch_x)
        loss = loss_function(outputs, batch_x)
        loss.backward()
        optimizer.step()
    # print the loss
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))

RuntimeError: expected scalar type Long but found Float

In [6]:
outputs.shape

torch.Size([64, 1, 784])